In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [ ]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "tanh")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "rect")
    caches.append(cache)
    
    return AL, caches

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    Z = W.dot(A_prev) + b
    linear_cache = (A_prev, W, b)     
    if activation == "tanh":
        A = np.tanh(Z)
    elif activation == "rect":
        A = np.array(Z, copy=True)
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [ ]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = (AL - Y)
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "rect")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "tanh")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [ ]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache    
    if activation == "tanh":
        dZ = dA * (1 - np.power(np.tanh(activation_cache),2))
    if activation == "rect":
        dZ = dA * 1
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db

In [ ]:
def L_layer_model(X, Y, x_val, y_val, layers_dims, learning_rate = 0.01, num_iterations = 1000):
    np.random.seed(1)
    costs = []
    costs_val = []
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):
        for j in range(0, (Y.shape[1])):
            k = np.random.randint(Y.shape[1])
            AL, caches =  L_model_forward(X[:, k].reshape(-1,1), parameters)
            grads = L_model_backward(AL, Y[:, k].reshape(-1,1), caches)
            parameters = update_parameters(parameters, grads, learning_rate)
        
        AL, caches =  L_model_forward(X, parameters)
        cost = (1/2) * np.sum(np.power(AL - Y, 2)) / (Y.shape[1])
        cost = np.squeeze(cost) 
        costs.append(cost)
        
        AL_val, _ = L_model_forward(x_val, parameters)
        cost_val = (1/2) * np.sum(np.power(AL_val - y_val, 2)) / (y_val.shape[1])
        cost_val = np.squeeze(cost_val) 
        costs_val.append(cost_val)
        if i % 250 == 0:
            print ("Iteración %i. Costo training: %f. Costo testing: %f" %(i, cost, cost_val))
        
    plt.figure(figsize=(8,6))
    plt.plot(np.squeeze(costs), 'blue', label = "Error training")
    plt.plot(np.squeeze(costs_val), 'red', label = "Error testing")
    plt.ylabel('Costo')
    plt.xlabel('Iteración')
    plt.legend(loc=0)
    plt.title("Learning rate =" + str(learning_rate))
    
    plt.show()
    
    return parameters

In [ ]:
from sklearn.model_selection import train_test_split

x = np.random.uniform(size = [500]) * 2 * np.pi
y = np.random.uniform(size = [500]) * 2 * np.pi
z = np.random.uniform(size = [500]) * 2 - 1
entrada = np.array([x, y, z])
f = (np.sin(x) + np.cos(y) + z).reshape(1,-1)

train_x, test_x, train_y, test_y = train_test_split(entrada.T, f.T, test_size=0.3)
train_x = train_x.T
train_y = train_y.T
test_x = test_x.T
test_y = test_y.T

print(entrada.shape)
print(f.shape)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
layers_dims = [train_x.shape[0], 16, 1]
parameters = L_layer_model(train_x, train_y, test_x, test_y, layers_dims, learning_rate = 0.01, num_iterations = 1001)

In [ ]:
pred_train, _ = L_model_forward(train_x, parameters)
indices00 = np.argsort(train_x[0, :])
indices01 = np.argsort(train_x[1, :])
indices02 = np.argsort(train_x[2, :])

pred_test, _ = L_model_forward(test_x, parameters)
indices10 = np.argsort(test_x[0, :])
indices11 = np.argsort(test_x[1, :])
indices12 = np.argsort(test_x[2, :])

plt.figure(figsize=(15, 10))

plt.subplot(2,3,1)
plt.plot(train_x[0, :][indices00], train_y[0][indices00], 'red')
plt.plot(train_x[0, :], pred_train[0], '*')
plt.title("Error de X en training")

plt.subplot(2,3,2)
plt.plot(train_x[1, :][indices01], train_y[0][indices01], 'green')
plt.plot(train_x[1, :], pred_train[0], '*')
plt.title("Error de Y en training")

plt.subplot(2,3,3)
plt.plot(train_x[2, :][indices02], train_y[0][indices02], 'blue')
plt.plot(train_x[2, :], pred_train[0], '*')
plt.title("Error de Z en training")

plt.subplot(2,3,4)
plt.plot(test_x[0, :][indices10], test_y[0][indices10], 'red')
plt.plot(test_x[0, :], pred_test[0], '*')
plt.title("Error de X en testing")

plt.subplot(2,3,5)
plt.plot(test_x[1, :][indices11], test_y[0][indices11], 'green')
plt.plot(test_x[1, :], pred_test[0], '*')
plt.title("Error de Y en testing")

plt.subplot(2,3,6)
plt.plot(test_x[2, :][indices12], test_y[0][indices12], 'blue')
plt.plot(test_x[2, :], pred_test[0], '*')
plt.title("Error de Z en testing")

plt.show()